In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

h2o.init(nthreads=-1, max_mem_size=12)

data = h2o.import_file('../input/training_dataset.csv')
pred = h2o.import_file('../input/test_dataset.csv')
data = data.drop(['id'], axis=1)

splits = data.split_frame(ratios=[0.8], seed=1)
train = splits[0]
test = splits[1]

y = 'scribe'
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

X_test = pred.drop(['id'], axis=1)

aml = rf(ntrees = 500, seed=1, nfolds=0)
aml.train(y=y, training_frame=train, validation_frame=test)

y_pred = aml.predict(X_test)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.5-ea" 2019-10-15; OpenJDK Runtime Environment (build 11.0.5-ea+10-post-Ubuntu-0ubuntu1); OpenJDK 64-Bit Server VM (build 11.0.5-ea+10-post-Ubuntu-0ubuntu1, mixed mode, sharing)
  Starting server from /home/ovi/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphbahajc6
  JVM stdout: /tmp/tmphbahajc6/h2o_ovi_started_from_python.out
  JVM stderr: /tmp/tmphbahajc6/h2o_ovi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_ovi_8cmbkx
H2O cluster total nodes:,1
H2O cluster free memory:,12 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [2]:
y_pred.head()

predict,Begonius,FerrumEffractarius,Franciscus,Marcus,Noaelius,Paithonius,Philippus,Ubuntius
Franciscus,0,0.00328343,0.827945,0.0319806,0,0.121912,0.010939,0.00394011
Ubuntius,0,0,0,0.0984368,0,0.0240456,0.10436,0.773158
Noaelius,0,0,0.0216535,0,0.978346,0,0,0
Marcus,0,0,0.00207991,0.964322,0,0,0.0332197,0.000378165
Marcus,0,1.32015e-05,0.00574924,0.812173,0,0.0210806,0.160984,0
Marcus,0,0,0.000404994,0.92927,0,0.00404994,0.0662749,0
Marcus,0,0,0.00846447,0.625936,0,0.00846447,0.331741,0.0253934
Philippus,0,0,0.0376423,0.0473315,0,0.00204204,0.889081,0.0239034
Noaelius,0,0,0.311936,0.0303341,0.637465,0.0112348,0.00453594,0.00449394
Marcus,0,0,0,0.952756,0,0.00197376,0.0452366,3.40304e-05


In [3]:
y_pred['predict']

predict
Franciscus
Ubuntius
Noaelius
Marcus
Marcus
Marcus
Marcus
Philippus
Noaelius
Marcus


In [4]:

submit = pred['id']
submit['scribe'] = y_pred['predict']
submit = submit.as_data_frame(use_pandas=True)

submit.to_csv('../output/submit_aqua_rf.csv', index=False)